In [ ]:
from scipy.io import loadmat
import mdtraj as md

### Generate traj, pdb, and weights for mtsl, basl, and ma-proxyl

In [ ]:
for label,resname,pos in zip(['R1A','BAS','MA1'],['MTSSL','BASL','MAP'],[29,27,37]):
    data = loadmat(f'rotamer_libraries/rotamer_library_{label}.mat', 
                   struct_as_record=False, squeeze_me=True)
    rot_lib = data['rot_lib']

    top = md.Topology()
    chain = top.add_chain()
    res = top.add_residue(resname, chain)
    cnt = 1
    for at_name,at_num in zip(data['rot_lib'].atom_tags,data['rot_lib'].elements):
        
        if cnt == pos:
            print(at_name)
            at_name = 'N1'
            print(at_name)
        if cnt == pos+1:
            print(at_name)
            at_name = 'O1'
            print(at_name)
            
        top.add_atom(at_name, element=md.element.Element.getByAtomicNumber(at_num), residue=res)
        cnt += 1
    xyz = np.array([data['rot_lib'].rotamers[i].coor for i in range(len(data['rot_lib'].rotamers))])
    unitcell_lengths = np.full((xyz.shape[0],3),10)
    unitcell_angles = np.full((xyz.shape[0],3),90)
    time = np.arange(xyz.shape[0])
    traj = md.Trajectory(xyz/10, top, time, unitcell_lengths, unitcell_angles)
    traj.save_dcd(f'rotamer_libraries/{label}.dcd')
    traj[0].save_pdb(f'rotamer_libraries/{label}.pdb')

    np.savetxt(f'rotamer_libraries/{label}_weights.txt',rot_lib.populations)